### Lexicon에 있는 키워드로 typography 이미지 크롤링

In [1]:
### import packages ###
import numpy as np
import pandas as pd
import time
import requests
import urllib.request
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import unicodedata

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
### data load ###
path = "C:/Users/ASUX/Desktop/컨퍼런스/Lexicon_data.txt"
data = pd.read_csv(path, names=["word", "emotion", "association"], sep='\t')
data.head()

,word,emotion,association
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0


In [3]:
### search list ###
angry = data[(data.association == 1) & (data.emotion == 'anger')].word.to_list()
anticipation = data[(data.association == 1) & (data.emotion == 'anticipation')].word.to_list()
disgust = data[(data.association == 1) & (data.emotion == 'disgust')].word.to_list()
fear = data[(data.association == 1) & (data.emotion == 'fear')].word.to_list()
joy = data[(data.association == 1) & (data.emotion == 'joy')].word.to_list()
negative = data[(data.association == 1) & (data.emotion == 'negative')].word.to_list()
positive = data[(data.association == 1) & (data.emotion == 'positive')].word.to_list()
sadness = data[(data.association == 1) & (data.emotion == 'sadness')].word.to_list()
surprise = data[(data.association == 1) & (data.emotion == 'surprise')].word.to_list()
trust = data[(data.association == 1) & (data.emotion == 'trust')].word.to_list()

emotion = list(data.emotion.unique())
text = ['typography', 'typeface', 'font']

In [6]:
### setting ###
# 이미지를 저장할 'img' 디렉토리를 미리 생성
path = 'C:/Users/ASUX/Desktop/chromedriver_win32/chromedriver'
SCROLL_NUM = 5
CRAWLING_NUM = 50

In [7]:
def get_url(driver, emotion, text):
    url = "https://www.google.co.kr/search?q=" + str(emotion) + " " + str(text)
    driver.get(url)
    driver.find_element_by_class_name('hide-focus-ring').click()
    time.sleep(3)
    
    html = driver.page_source
    soup = bs(html, 'lxml')

In [8]:
def scroll(drive, scroll_num):
    
    for i in range(scroll_num):
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(2)
        
    driver.execute_script('window.scrollTo(0,0);')

In [9]:
def crawl(driver, crawl_num, emotion, text):
    
    x = 1
    n = 1
    driver.find_elements_by_css_selector(".bRMDJf.islir")[0].click()
    time.sleep(1.5)
    
    for i in range(crawl_num):
        img_url = driver.find_elements_by_class_name('n3VNCb')[0].get_property('src')
        image_name = emotion + ' ' + text + '_' + str(n) + '.jpg'
                    
        try:
            with urllib.request.urlopen(img_url) as f:
                with open('./img/' + emotion + ' ' + text + '_' + str(n) + '.jpg', 'wb') as h:
                    img = f.read()
                    h.write(img)

            # data_frame 저장용
            image_url_list.append(img_url)
            image_name_list.append(image_name)

            driver.find_elements_by_class_name("CIF8af")[x].click() # 다음 이미지 선택
            time.sleep(2)
            x = 3
            n += 1

        except:
            driver.find_elements_by_class_name("CIF8af")[x].click() # 다음 이미지 선택
            time.sleep(2)
            x = 3
            n += 1

In [10]:
### crawling ###
image_name_list = []
image_url_list = []

driver = webdriver.Chrome(path)
driver.maximize_window()
driver.get('https://www.google.co.kr/')
time.sleep(1)

for emo in emotion: # search list에서 인덱싱
    for t in text:
        get_url(driver, emo, t)
        time.sleep(2)
        scroll(driver, SCROLL_NUM)
        time.sleep(2)
        crawl(driver, CRAWLING_NUM, emo, t)

In [13]:
##### save ###
data = {'image_name':image_name_list,
        'image_url':image_url_list}

df = pd.DataFrame(data)
df.head()
#df.to_csv('data.csv', index=True, encoding='utf-8-sig')

,image_name,image_url
0,anger typography_1.jpg,https://i.pinimg.com/originals/17/61/eb/1761eb...
1,anger typography_2.jpg,https://i.pinimg.com/originals/17/61/eb/1761eb...
2,anger typography_3.jpg,https://fontesk.com/wp-content/uploads/2020/10...
3,anger typography_4.jpg,https://previews.123rf.com/images/ponytail1414...
4,anger typography_5.jpg,https://pro2-bar-s3-cdn-cf4.myportfolio.com/5a...
